# 🔄 04. 반복 자동화 & 여러 시트 한번에 처리

> **목표**: `for`문과 함수를 결합하여 **여러 시트**, **여러 파일**을 한번에 자동 처리한다.
>
> **왜 중요한가?** — "10개 시트 다 똑같이 해줘"라는 요청. 하나씩 하면 30분, 자동화하면 **3초**.

---

### 📚 이 챕터에서 배우는 것

| 순서 | 주제 | 실무 적용 |
|:---:|:---|:---|
| 1 | `expand()` — 동적 범위 | 데이터가 몇 행인지 자동 감지 |
| 2 | 모든 시트 순회 | 시트별 동일 작업 일괄 처리 |
| 3 | 동적 합계 행 추가 | 데이터 끝에 자동 합계 |
| 4 | 조건부 셀 강조 | 기준 미달 셀 빨간색 표시 |
| 5 | 행/열 삽입/삭제 | 빈 행 자동 정리 |
| 6 | 여러 파일 일괄 처리 | 폴더 내 모든 xlsx 처리 |
| 7 | 🔥 실전 종합 | 전 시트 보고서 포맷 일괄 적용 |

---
## 🧠 핵심 이론: `expand()` — 동적 범위의 핵심

실무 데이터는 행 수가 매번 다릅니다. `A1:A100` 하드코딩하면 안 됩니다!

| 코드 | 동작 |
|:---|:---|
| `ws.range('A1').expand()` | 표 전체 (아래+오른쪽) |
| `ws.range('A1').expand('down')` | 아래로만 |
| `ws.range('A1').expand('right')` | 오른쪽으로만 |
| `ws.range('A1').expand('table')` | 표 전체 (= expand()) |

> ⚠️ **주의**: `expand()`는 빈 셀을 만나면 멈춥니다!  
> 중간에 빈 행이 있으면 거기서 끊깁니다.

---
## 0️⃣ 준비: 엑셀 파일 열기

In [ ]:
import xlwings as xw

app = xw.App(add_book=False)
wb = app.books.open('교육이수현황.xlsx')

print(f'✅ 파일 열기 완료: {wb.name}')
print(f'📋 시트 목록: {[s.name for s in wb.sheets]}')

---
## 1️⃣ 동적 범위 탐색: `expand()` 활용

In [ ]:
ws = wb.sheets['마케팅팀']

# 마지막 행 번호 구하기
last_row = ws.range('B2').expand('down').last_cell.row
print(f'📍 마지막 행: {last_row}행')

# 마지막 열 번호 구하기
last_col = ws.range('A2').expand('right').last_cell.column
print(f'📍 마지막 열: {last_col}열')

In [ ]:
# 전체 데이터 범위 가져오기 (테이블 전체)
all_data = ws.range('A2').expand('table').value

print(f'📊 전체 데이터:')
for row in all_data:
    print(f'  {row}')

In [ ]:
# 범위의 주소를 문자열로 확인하기
data_range = ws.range('A2').expand('table')
print(f'📍 데이터 범위 주소: {data_range.address}')
print(f'📍 시작 셀: {data_range.first_cell.address}')
print(f'📍 끝 셀: {data_range.last_cell.address}')

---
## 2️⃣ 모든 시트 순회하기

In [ ]:
# 방법 1: 모든 시트 이름 출력
print('📋 전체 시트 목록:')
for sheet in wb.sheets:
    print(f'  - {sheet.name}')

In [ ]:
# 방법 2: 인덱스로 접근
print('📋 인덱스별 시트:')
for i in range(len(wb.sheets)):
    ws_temp = wb.sheets[i]
    print(f'  [{i}] {ws_temp.name}')

In [ ]:
# 방법 3: 특정 시트만 필터링해서 처리
target_sheets = ['영업1팀', '영업2팀', '마케팅팀']

all_sheet_names = [s.name for s in wb.sheets]

for name in target_sheets:
    if name in all_sheet_names:
        ws_temp = wb.sheets[name]
        print(f'  ✅ {ws_temp.name} → 처리 대상')
    else:
        print(f'  ⏭️ {name} → 시트 없음, 건너뜀')

---
## 3️⃣ 모든 시트에 헤더 서식 일괄 적용

In [ ]:
def apply_header_style(ws):
    """모든 시트의 2행(헤더)에 동일한 서식 적용"""
    # 헤더 범위 자동 감지
    header = ws.range('A2').expand('right')
    
    # 데이터가 없으면 건너뜀
    if header.value is None:
        print(f'  ⏭️ {ws.name}: 데이터 없음, 건너뜀')
        return
    
    header.font.bold = True
    header.font.size = 11
    header.font.name = '맑은 고딕'
    header.font.color = (255, 255, 255)
    header.color = (68, 114, 196)
    header.api.HorizontalAlignment = -4108
    
    print(f'  ✅ {ws.name}: 헤더 서식 적용 완료')


print('✅ apply_header_style() 함수 정의 완료')

In [ ]:
# 모든 시트에 한번에 적용!
print('🔄 전체 시트 헤더 서식 적용 시작...')

for sheet in wb.sheets:
    try:
        apply_header_style(sheet)
    except Exception as e:
        print(f'  ⚠️ {sheet.name}: {e}')

print('\n🎉 완료!')

---
## 4️⃣ 동적 합계 행 자동 추가

> 데이터가 몇 행이든 상관없이, **마지막 행 아래에 자동으로 합계**를 넣습니다.

In [ ]:
def add_total_row(ws, data_col='C', label_col='B', start_row=3):
    """
    데이터 마지막 행 다음에 합계 행 추가
    
    Parameters:
        ws: 워크시트
        data_col: 합계를 구할 열 (기본: C)
        label_col: '합계' 라벨 열 (기본: B)
        start_row: 데이터 시작 행 (기본: 3)
    """
    # 마지막 데이터 행 찾기
    cell = ws.range(f'{data_col}{start_row}')
    if cell.value is None:
        print(f'  ⏭️ {ws.name}: {data_col}열 데이터 없음')
        return
    
    last_row = cell.expand('down').last_cell.row
    total_row = last_row + 1
    
    # 합계 라벨
    ws.range(f'{label_col}{total_row}').value = '합계'
    ws.range(f'{label_col}{total_row}').font.bold = True
    
    # SUM 수식 입력
    ws.range(f'{data_col}{total_row}').value = f'=SUM({data_col}{start_row}:{data_col}{last_row})'
    ws.range(f'{data_col}{total_row}').font.bold = True
    ws.range(f'{data_col}{total_row}').color = (255, 242, 204)  # 연한 노랑
    
    print(f'  📊 {ws.name}: SUM({data_col}{start_row}:{data_col}{last_row}) → {data_col}{total_row}')


print('✅ add_total_row() 함수 정의 완료')

In [ ]:
# 마케팅팀 시트에 합계 추가 테스트
ws = wb.sheets['마케팅팀']
add_total_row(ws, data_col='C', label_col='B', start_row=3)

print('\n💡 엑셀에서 합계가 추가됐는지 확인해보세요!')

---
## 5️⃣ 조건부 셀 강조 (교육시간 기준 미달 표시)

> 교육 시간이 **10시간 미만**이면 빨간색, **이상이면** 초록색으로 표시

In [ ]:
def highlight_low_hours(ws, col='C', start_row=3, threshold=10):
    """
    교육 시간이 기준 미만인 셀을 강조
    
    Parameters:
        ws: 워크시트
        col: 검사할 열
        start_row: 데이터 시작 행
        threshold: 기준값 (미만이면 빨강)
    """
    cell = ws.range(f'{col}{start_row}')
    if cell.value is None:
        print(f'  ⏭️ {ws.name}: 데이터 없음')
        return
    
    last_row = cell.expand('down').last_cell.row
    count_low = 0
    count_ok = 0
    
    for row in range(start_row, last_row + 1):
        value = ws.range(f'{col}{row}').value
        
        if value is not None and isinstance(value, (int, float)):
            if value < threshold:
                ws.range(f'{col}{row}').color = (255, 199, 206)      # 연한 빨강
                ws.range(f'{col}{row}').font.color = (156, 0, 6)     # 진한 빨강 글자
                ws.range(f'{col}{row}').font.bold = True
                count_low += 1
            else:
                ws.range(f'{col}{row}').color = (198, 239, 206)      # 연한 초록
                ws.range(f'{col}{row}').font.color = (0, 97, 0)      # 진한 초록 글자
                count_ok += 1
    
    print(f'  ✅ {ws.name}: 정상 {count_ok}건, 미달 {count_low}건')


print('✅ highlight_low_hours() 함수 정의 완료')

In [ ]:
# 마케팅팀에 조건부 강조 적용
ws = wb.sheets['마케팅팀']
highlight_low_hours(ws, col='C', start_row=3, threshold=10)

print('\n💡 10시간 미만 셀이 빨간색으로 표시됐는지 확인해보세요!')

---
## 6️⃣ 행/열 삽입/삭제

> ⚠️ **핵심 규칙**: 행 삭제 시 **반드시 아래→위(역순)**로 해야 합니다!  
> 위→아래로 하면 행 번호가 밀려서 엉뚱한 행이 삭제됩니다.

In [ ]:
# 행 삽입 (3행 위에 새 행 삽입)
# ws.range('3:3').api.Insert()
# print('✅ 3행 위에 새 행 삽입 완료')

# 열 삽입 (C열 왼쪽에 새 열 삽입)
# ws.range('C:C').api.Insert()
# print('✅ C열 왼쪽에 새 열 삽입 완료')

print('💡 행/열 삽입 코드는 주석 해제 후 실행하세요 (데이터 구조가 바뀝니다!)')

In [ ]:
# 🎯 실무 패턴: 빈 행 찾아서 삭제하기
# ★ 반드시 역순으로! (아래 → 위)

def delete_empty_rows(ws, col='A', start_row=2):
    """
    특정 열 기준으로 빈 행을 삭제
    반드시 역순(아래→위)으로 삭제해야 안전!
    """
    last_row = ws.cells.last_cell.row  # 시트의 마지막 사용 행
    deleted = 0
    
    for row in range(last_row, start_row - 1, -1):   # ★ 역순!
        if ws.range(f'{col}{row}').value is None:
            ws.range(f'{row}:{row}').api.Delete()
            deleted += 1
    
    print(f'  🗑️ {ws.name}: {deleted}개 빈 행 삭제')


print('✅ delete_empty_rows() 함수 정의 완료')
print('💡 사용: delete_empty_rows(ws, col="A", start_row=2)')

---
## 7️⃣ 여러 엑셀 파일 일괄 처리

> 폴더 안에 있는 **모든 `.xlsx` 파일**을 한번에 처리하는 패턴

In [ ]:
import os
from pathlib import Path

def process_all_excel_files(folder_path):
    """폴더 내 모든 xlsx 파일을 찾아서 처리"""
    
    app_local = xw.App(add_book=False, visible=False)  # 화면에 안 보이게
    
    folder = Path(folder_path)
    excel_files = list(folder.glob('*.xlsx'))
    
    print(f'📁 {folder_path}')
    print(f'📄 {len(excel_files)}개 엑셀 파일 발견\n')
    
    for filepath in excel_files:
        print(f'📄 처리 중: {filepath.name}')
        
        try:
            wb_temp = app_local.books.open(str(filepath))
            
            for sheet in wb_temp.sheets:
                # 각 시트에 원하는 작업 수행
                apply_header_style(sheet)
            
            wb_temp.save()
            wb_temp.close()
            
        except Exception as e:
            print(f'  ❌ 에러: {e}')
    
    app_local.quit()
    print(f'\n🎉 전체 처리 완료!')


print('✅ process_all_excel_files() 함수 정의 완료')
print('💡 사용: process_all_excel_files(r"C:\\내폴더")')

In [ ]:
# 현재 폴더의 xlsx 파일 목록만 먼저 확인
from pathlib import Path

current_folder = Path('.')
excel_files = list(current_folder.glob('*.xlsx'))

print(f'📁 현재 폴더의 엑셀 파일:')
for f in excel_files:
    size_kb = f.stat().st_size / 1024
    print(f'  📄 {f.name} ({size_kb:.1f} KB)')

---
## 🔥 8. 실전 종합: 전 시트 보고서 포맷 일괄 적용

> 03장에서 만든 서식 함수 + 04장의 반복문을 합쳐서,  
> **교육이수현황 파일의 모든 시트에 보고서 서식을 한번에 적용**합니다.

In [ ]:
def set_border(ws, cell_range, style=1, weight=2):
    """테두리 설정 헬퍼 함수"""
    rng = ws.range(cell_range).api
    for border_id in range(7, 13):
        try:
            rng.Borders(border_id).LineStyle = style
            rng.Borders(border_id).Weight = weight
        except:
            pass


def full_format_all_sheets(wb):
    """교육이수현황 파일의 모든 시트에 보고서 서식 일괄 적용"""
    
    for sheet in wb.sheets:
        ws = sheet
        print(f'\n{"="*40}')
        print(f'📋 {ws.name} 시트 처리 시작')
        print(f'{"="*40}')
        
        # 데이터 유무 확인
        if ws.range('A2').value is None and ws.range('B2').value is None:
            print(f'  ⏭️ 빈 시트, 건너뜀')
            continue
        
        # 1) 헤더 범위 감지
        try:
            header_range = ws.range('A2').expand('right')
            last_col_letter = header_range.last_cell.address.split('$')[1]
            header_str = f'A2:{last_col_letter}2'
            print(f'  📍 헤더 범위: {header_str}')
        except:
            print(f'  ⚠️ 헤더 감지 실패')
            continue
        
        # 2) 데이터 범위 감지
        try:
            data_range = ws.range('A2').expand('table')
            last_data_row = data_range.last_cell.row
            data_str = f'A2:{last_col_letter}{last_data_row}'
            print(f'  📍 데이터 범위: {data_str}')
        except:
            print(f'  ⚠️ 데이터 범위 감지 실패')
            continue
        
        # 3) 제목
        title_range = f'A1:{last_col_letter}1'
        ws.range(title_range).merge()
        ws.range('A1').value = f'📊 {ws.name} 교육이수현황'
        ws.range('A1').font.size = 16
        ws.range('A1').font.bold = True
        ws.range('A1').font.name = '맑은 고딕'
        ws.range('A1').font.color = (255, 255, 255)
        ws.range('A1').color = (47, 85, 151)
        ws.range(title_range).api.HorizontalAlignment = -4108
        ws.range('A1').row_height = 45
        print(f'  ✅ 제목 적용')
        
        # 4) 헤더 스타일
        ws.range(header_str).font.bold = True
        ws.range(header_str).font.size = 11
        ws.range(header_str).font.name = '맑은 고딕'
        ws.range(header_str).font.color = (255, 255, 255)
        ws.range(header_str).color = (68, 114, 196)
        ws.range(header_str).api.HorizontalAlignment = -4108
        ws.range('A2').row_height = 30
        print(f'  ✅ 헤더 서식')
        
        # 5) 테두리
        set_border(ws, data_str)
        print(f'  ✅ 테두리')
        
        # 6) 줄무늬
        for row in range(3, last_data_row + 1):
            if row % 2 == 1:
                ws.range(f'A{row}:{last_col_letter}{row}').color = (221, 235, 247)
        print(f'  ✅ 줄무늬')
        
        # 7) 열 자동 맞춤
        ws.range(data_str).columns.autofit()
        print(f'  ✅ 열 너비 자동 조정')
    
    print(f'\n{"="*40}')
    print(f'🎉 전체 시트 서식 적용 완료!')
    print(f'{"="*40}')


print('✅ full_format_all_sheets() 함수 정의 완료!')

In [ ]:
# 실행!  모든 시트에 보고서 서식 일괄 적용
full_format_all_sheets(wb)

# 저장
wb.save()
print('\n💾 파일 저장 완료!')

---
## 💾 마무리

In [ ]:
# 엑셀 닫기 (확인 후 실행)
# app.quit()

print('💡 작업 확인 후 app.quit() 실행하세요')

---
## ⚠️ 흔한 에러와 해결

| 증상 | 원인 | 해결 |
|:---|:---|:---|
| `expand()`가 1개만 리턴 | 시작 셀에 값이 없거나 빈 행이 중간에 있음 | 시작 셀 확인 |
| 행 삭제 후 데이터가 이상함 | 위→아래 순서로 삭제함 | **반드시 역순** (아래→위) |
| `visible=False`인데 엑셀 떠있음 | 이전 실행에서 `app.quit()` 안 함 | 작업 관리자에서 EXCEL.EXE 종료 |
| 파일이 이미 열려있다는 에러 | 다른 프로세스가 사용 중 | 파일 닫고 재시도 |
| `for sheet`에서 순서가 꼬임 | 순회 중 시트 추가/삭제 | 시트 목록을 `list()`로 먼저 복사 |

---
## 📝 연습 문제

1. 모든 시트에 `highlight_low_hours()` 함수를 적용해 보세요
2. 모든 시트에서 **빈 행을 자동 삭제**하는 코드를 작성해 보세요
3. 폴더 내 **모든 `.xlsx` 파일 이름과 시트 수**를 출력하는 코드를 만들어 보세요

In [ ]:
# 🏋️ 연습 1: 모든 시트에 조건부 강조 적용
# 아래 코드를 완성해보세요!

# for sheet in wb.sheets:
#     highlight_low_hours(???, col='C', threshold=???)

---
## 📌 다음 챕터 예고

> **05. 데이터 취합 & 복사 붙여넣기**  
> 여러 시트/파일의 데이터를 하나의 시트로 합치고, 서식까지 보존하는 방법